In [1]:
import mbuild as mb
import mdtraj as md
import nglview
from foyer.forcefield import Forcefield as FoyerForcefield
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff.forcefield import ForceField
from openff.units import unit

from openff.interchange.components.interchange import Interchange
from openff.interchange.components.mbuild import offmol_to_compound
from openff.interchange.components.mdtraj import _combine_topologies, _OFFBioTop
from openff.interchange.drivers import get_openmm_energies

In [2]:
nanoparticle = mb.load("silica_nanoparticle.json")

In [3]:
oplsaa_silica = FoyerForcefield("oplsaa_switchable.xml")

/Users/mwt/miniconda3/envs/openff-interchange-env/lib/python3.9/site-packages/foyer/forcefield.py:656: UserWarning: Non-atomistic element type detected. Creating custom element for _EP
  warnings.warn(
/Users/mwt/miniconda3/envs/openff-interchange-env/lib/python3.9/site-packages/foyer/forcefield.py:620: UserWarning: No force field version number found in force field XML file.
  warnings.warn(
/Users/mwt/miniconda3/envs/openff-interchange-env/lib/python3.9/site-packages/foyer/forcefield.py:632: UserWarning: No force field name found in force field XML file.
  warnings.warn(
/Users/mwt/miniconda3/envs/openff-interchange-env/lib/python3.9/site-packages/foyer/forcefield.py:644: UserWarning: No combining rule found in force field XML file.
  warnings.warn(
/Users/mwt/miniconda3/envs/openff-interchange-env/lib/python3.9/site-packages/foyer/validator.py:165: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)


In [4]:
topology = _OFFBioTop(mdtop=md.Topology.from_openmm(nanoparticle.to_parmed().topology))

In [5]:
nanoparticle_interchange = Interchange.from_foyer(
    topology=topology,
    force_field=oplsaa_silica,
)

In [6]:
n_solvent = 800

solvent = Molecule.from_smiles("C1CCOC1")
solvent.generate_conformers(n_conformers=1)

In [7]:
sage = ForceField("openff_unconstrained-2.0.0.offxml")

In [8]:
solvent_interchange = Interchange.from_smirnoff(
    sage,
    _OFFBioTop(
        mdtop=md.Topology.from_openmm(
            Topology.from_molecules(n_solvent * [solvent]).to_openmm()
        )
    ),
)

In [9]:
combined = solvent_interchange + nanoparticle_interchange

/Users/mwt/software/openff-system/openff/interchange/components/interchange.py:617: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this method is not guaranteed to be suitable for production. Use with extreme caution and thoroughly validate results!
  warnings.warn(
/Users/mwt/software/openff-system/openff/interchange/components/interchange.py:649: UserWarning: 'other' Interchange object has handler with name RBTorsions not found in 'self,' but it has now been added.
  warnings.warn(
/Users/mwt/software/openff-system/openff/interchange/components/interchange.py:649: UserWarning: 'other' Interchange object has handler with name RBImpropers not found in 'self,' but it has now been added.
  warnings.warn(
/Users/mwt/software/openff-system/openff/interchange/components/interchange.py:672: UserWarning: Setting positions to None because one or both objects added together were missing positions.
  warnings.warn(


In [10]:
thf_compound = offmol_to_compound(solvent)

In [11]:
solvated_compound = mb.packing.solvate(
    solute=nanoparticle, solvent=thf_compound, n_solvent=n_solvent, box=mb.Box(3 * [8])
)

In [12]:
combined.topology = _combine_topologies(
    solvent_interchange.topology,
    nanoparticle_interchange.topology,
)
combined.positions = solvated_compound.xyz * unit.nanometer
combined.box = 3 * [8] * unit.nanometer

In [13]:
combined.to_pdb("out.pdb")
view = nglview.show_mdtraj(md.load("out.pdb"))
view.clear_representations()
view.add_representation(
    "spacefill", selection=[*range(combined.topology.mdtop.n_atoms)]
)
view

NGLWidget()

In [14]:
get_openmm_energies(combined, combine_nonbonded_forces=True)

EnergyReport(energies={'Bond': <Quantity(9.71710874e+09, 'kilojoule / mole')>, 'Angle': <Quantity(1303887.88, 'kilojoule / mole')>, 'Torsion': <Quantity(0.0, 'kilojoule / mole')>, 'Nonbonded': <Quantity(0.0, 'kilojoule / mole')>})